In [1]:
"""
Developed by Shayan Sepahvand, December 2023, Toronto Met. Univ.
"""
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import PIL.Image

2024-01-18 20:21:38.058574: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-18 20:21:38.086277: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Load the images

In [2]:
def load_images_from_folder(directory):
    images = []
    fileslist = os.listdir(directory)
    fileslist = sorted(fileslist , key = lambda fileslist: int(fileslist.split('.')[0]))
    for filename in fileslist:
        #img = cv.imread(os.path.join(directory,filename))
        loc = os.path.join(directory,filename)
        img = tf.keras.preprocessing.image.load_img(loc, target_size=(300, 300))
        img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        if img is not None:
            images.append(img)
    return np.asarray(images)

Load the angles (labels)

In [3]:
def load_angles_from_folder(directory):
    with open(directory,'r')  as ff:
        content = ff.readline()
        angles = content.split(sep='\t') #lines are separated by tabs
        angles = np.asarray(angles, dtype = np.float32) * (np.pi/180)
        return angles

Load the images and angles

In [4]:
directory_imgs = './gauss/'
directory_angles = './Dataset/Labels.txt'
imgs = load_images_from_folder(directory_imgs)
angles = load_angles_from_folder(directory_angles)

Define the Testing Datasets

In [5]:
testing_imgs = imgs[900:]
testing_angles = angles[900:]

Load the trained CNN

In [8]:
model = tf.keras.models.load_model('./TrainedCNNs/for_test_guass_400epochs')
# model.summary() 
# model.get_config()

Evaluate the model for test dataset

In [9]:
loss = model.evaluate(testing_imgs , testing_angles, batch_size = 16)

7/7 [==============================] - 1s 69ms/step - loss: 0.0981 - mean_absolute_error: 0.2642


Tries to load several image from the test dataset and determine the estimated angle versus the real angle

In [ ]:
def return_estimated_angles():
    predicted_angles_list = []
    for testimage in testing_imgs:
        test_img = np.expand_dims(testimage , axis = 0)
        predicted_angles = model.predict(test_img , verbose = 0)
        predicted_angles_list.append(predicted_angles)


    return np.array(predicted_angles_list)

Let's plot the estimated angles vs real angles

In [ ]:
predicted_angles = return_estimated_angles()
predicted_angles = np.ndarray.flatten(predicted_angles)

In [ ]:
SampleNo = np.arange(1,len(testing_angles) + 1)
# Create a scatter plot for the first set of data (blue color)
plt.stem(SampleNo , predicted_angles, markerfmt='ro' , linefmt = '--')

# Create a scatter plot for the second set of data (red color)
plt.stem(SampleNo, testing_angles, markerfmt='bs' , linefmt = '--')


# Add labels and a legend
plt.xlabel('Sample Number')
plt.ylabel('Angle (rad)')
plt.legend(['Predicted Angles', 'Measured Angles'])
plt.title('Prediction Result for the Test Data')
plt.grid(visible=True)
plt.ylim((-5,5))
plt.xlim((0,100))

# Show the plot
plt.show()


Scatter Plot

In [ ]:
plt.scatter(SampleNo, predicted_angles, c = 'blue', marker= 'x')
plt.scatter(SampleNo, testing_angles, c = 'red', marker= 'o', alpha=0.7)
plt.xlabel('Sample Number')
plt.ylabel('Angle (rad)')
plt.legend(['Predicted Angles', 'Measured Angles'])
plt.title('Prediction Result for the Test Data')
plt.grid(visible=True)
plt.ylim((-5,5))
plt.xlim((0,100))
#plt.savefig('./Results_Paper/figures/scatter_dark.eps', format='eps')

Error Plot

In [ ]:
predition_error = (predicted_angles - testing_angles)
print(np.mean(abs(predition_error)))
print(np.max(abs(predition_error)))
print(np.min(abs(predition_error)))
font = {'weight' : 'bold',
        'size'   : 12}
plt.rc ('font', **font)
plt.stem(SampleNo , predition_error , markerfmt='ro' )
predition_error_abs = abs(predicted_angles - testing_angles)
MaxError = np.max(predition_error_abs) * np.ones(shape=(len(predition_error),))
plt.plot(SampleNo,   MaxError , color = 'pink')
MeanError = np.mean(predition_error_abs) * np.ones(shape=(len(predition_error),))
plt.plot(SampleNo,   MeanError , color = 'green')
plt.xlabel('Sample Number')
plt.ylabel('Predition Error (rad)')
#plt.legend(['Predicted Angles', 'Measured Angles'])
plt.title('Prediction Error for the Test Data')
plt.grid(visible = False)
plt.ylim((-2.1,2.2))
plt.xlim((0,100))
plt.legend(['Max Absolute Error', 'Mean Absolute Error', 'Prediction Error Angles'], loc = 'upper right')
#plt.savefig('./Results_Paper/figures/Prediction_Error_dark.eps', format='eps')

Save the absolute value of the error

In [ ]:
#np.save('./absolute_error/dark_error.npy', predition_error_abs)